# ML Observability & Monitoring Platform

**Comprehensive ML observability with monitoring tables, dashboards, alerting, and model drift detection**

## **Observability Objectives:**
1. **Model Performance Monitoring** - Real-time tracking of model accuracy, latency, and throughput
2. **Automated Alerting** - Proactive notifications for performance degradation and anomalies
3. **Interactive Dashboards** - Visual monitoring interfaces for stakeholders
4. **Model Drift Detection** - Statistical monitoring for data and concept drift
5. **Audit & Compliance** - Comprehensive logging for regulatory requirements

## **Observability Components:**
- **Monitoring Tables**: Centralized metrics storage and historical tracking
- **Drift Detection**: Statistical tests for input/output distribution changes
- **Alert System**: Configurable thresholds and notification channels
- **Executive Dashboards**: High-level KPIs and business metrics
- **Operational Dashboards**: Technical performance and health monitoring

**Prerequisites:** Run notebooks 05-08 first to establish the complete ML pipeline


In [ ]:
# Environment Setup for ML Observability
import sys
import os
import json
import datetime
import time
import numpy as np
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass

# Fix path for snowflake_connection module
current_dir = os.getcwd()
if "notebooks" in current_dir:
    src_path = os.path.join(current_dir, "..", "src")
else:
    src_path = os.path.join(current_dir, "src")

sys.path.append(src_path)
print(f"Added to Python path: {src_path}")

from snowflake_connection import get_session
from snowflake.snowpark.functions import (
    col, lit, when, count, avg, sum as sum_, max as max_, min as min_,
    stddev, variance, percentile_cont, corr, row_number, lag
)
from snowflake.snowpark.types import (
    StructType, StructField, StringType, DoubleType, IntegerType,
    FloatType, BooleanType, TimestampType, ArrayType
)
from snowflake.snowpark.window import Window

# Get Snowflake session
session = get_session()
print("SUCCESS: Environment ready for ML observability")
print("Capabilities: Model monitoring, drift detection, alerting, dashboards")
print("Tools: Statistical analysis, threshold monitoring, automated reporting")


In [ ]:
# Comprehensive Monitoring Infrastructure Setup (Fixed)
print("Setting up comprehensive ML monitoring infrastructure...")

# Create monitoring tables one by one to avoid multi-statement issues
monitoring_tables = [
    {
        "name": "ML_MODEL_PERFORMANCE_MONITORING",
        "sql": """
            CREATE TABLE IF NOT EXISTS ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_MODEL_PERFORMANCE_MONITORING (
                MONITORING_ID STRING,
                MODEL_NAME STRING,
                MODEL_VERSION STRING,
                METRIC_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
                
                -- Performance Metrics
                ACCURACY FLOAT,
                PRECISION_SCORE FLOAT,
                RECALL_SCORE FLOAT,
                F1_SCORE FLOAT,
                MAE FLOAT,
                RMSE FLOAT,
                R_SQUARED FLOAT,
                
                -- Operational Metrics
                PREDICTION_VOLUME INT,
                AVERAGE_RESPONSE_TIME_MS FLOAT,
                SUCCESS_RATE FLOAT,
                ERROR_RATE FLOAT,
                
                -- Data Quality Metrics
                MISSING_VALUES_PERCENTAGE FLOAT,
                OUTLIER_PERCENTAGE FLOAT,
                DATA_COMPLETENESS_SCORE FLOAT,
                
                -- Business Metrics
                HIGH_RISK_PREDICTIONS_COUNT INT,
                MEDIUM_RISK_PREDICTIONS_COUNT INT,
                LOW_RISK_PREDICTIONS_COUNT INT,
                
                -- Monitoring Metadata
                MONITORING_PERIOD_START TIMESTAMP_NTZ,
                MONITORING_PERIOD_END TIMESTAMP_NTZ,
                ENVIRONMENT STRING,
                STATUS STRING
            )
        """
    },
    {
        "name": "ML_MODEL_DRIFT_DETECTION",
        "sql": """
            CREATE TABLE IF NOT EXISTS ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_MODEL_DRIFT_DETECTION (
                DRIFT_ID STRING,
                MODEL_NAME STRING,
                DRIFT_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
                
                -- Drift Detection Results
                DRIFT_TYPE STRING,
                FEATURE_NAME STRING,
                DRIFT_SCORE FLOAT,
                P_VALUE FLOAT,
                DRIFT_THRESHOLD FLOAT,
                DRIFT_DETECTED BOOLEAN,
                DRIFT_SEVERITY STRING,
                
                -- Statistical Measures
                BASELINE_MEAN FLOAT,
                CURRENT_MEAN FLOAT,
                BASELINE_STD FLOAT,
                CURRENT_STD FLOAT,
                KS_STATISTIC FLOAT,
                
                -- Comparison Periods
                BASELINE_PERIOD_START TIMESTAMP_NTZ,
                BASELINE_PERIOD_END TIMESTAMP_NTZ,
                CURRENT_PERIOD_START TIMESTAMP_NTZ,
                CURRENT_PERIOD_END TIMESTAMP_NTZ,
                
                -- Action Required
                REQUIRES_RETRAINING BOOLEAN,
                REQUIRES_INVESTIGATION BOOLEAN,
                ALERT_SENT BOOLEAN DEFAULT FALSE
            )
        """
    },
    {
        "name": "ML_ALERT_MANAGEMENT",
        "sql": """
            CREATE TABLE IF NOT EXISTS ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_ALERT_MANAGEMENT (
                ALERT_ID STRING,
                ALERT_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
                
                -- Alert Details
                ALERT_TYPE STRING,
                ALERT_SEVERITY STRING,
                ALERT_TITLE STRING,
                ALERT_MESSAGE STRING,
                
                -- Source Information
                MODEL_NAME STRING,
                METRIC_NAME STRING,
                CURRENT_VALUE FLOAT,
                THRESHOLD_VALUE FLOAT,
                BASELINE_VALUE FLOAT,
                
                -- Alert Lifecycle
                ALERT_STATUS STRING DEFAULT 'ACTIVE',
                ACKNOWLEDGED_BY STRING,
                ACKNOWLEDGED_TIMESTAMP TIMESTAMP_NTZ,
                RESOLVED_BY STRING,
                RESOLVED_TIMESTAMP TIMESTAMP_NTZ,
                RESOLUTION_NOTES STRING,
                
                -- Notification
                NOTIFICATION_CHANNELS STRING,
                NOTIFICATION_SENT BOOLEAN DEFAULT FALSE,
                ESCALATION_LEVEL INT DEFAULT 1
            )
        """
    },
    {
        "name": "ML_BUSINESS_IMPACT_MONITORING", 
        "sql": """
            CREATE TABLE IF NOT EXISTS ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_BUSINESS_IMPACT_MONITORING (
                IMPACT_ID STRING,
                MONITORING_TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
                
                -- Clinical Impact Metrics
                PATIENTS_RISK_ASSESSED INT,
                HIGH_RISK_PATIENTS_IDENTIFIED INT,
                CLINICAL_INTERVENTIONS_TRIGGERED INT,
                POTENTIAL_ADVERSE_EVENTS_PREVENTED INT,
                
                -- Operational Impact
                COST_SAVINGS_ESTIMATED FLOAT,
                EFFICIENCY_IMPROVEMENT_PERCENTAGE FLOAT,
                STAFF_TIME_SAVED_HOURS FLOAT,
                
                -- Quality Metrics
                FALSE_POSITIVE_RATE FLOAT,
                FALSE_NEGATIVE_RATE FLOAT,
                CLINICAL_ACCURACY_FEEDBACK_SCORE FLOAT,
                
                -- Period Definition
                MEASUREMENT_PERIOD_START TIMESTAMP_NTZ,
                MEASUREMENT_PERIOD_END TIMESTAMP_NTZ,
                REPORTING_FREQUENCY STRING
            )
        """
    }
]

# Create tables individually
created_tables = []
for table in monitoring_tables:
    try:
        session.sql(table["sql"]).collect()
        created_tables.append(table["name"])
        print(f"   SUCCESS: {table['name']}")
    except Exception as e:
        print(f"   WARNING: {table['name']}: {e}")

print(f"\nSuccessfully created {len(created_tables)} monitoring tables:")
for table_name in created_tables:
    print(f"   - {table_name}")

print("SUCCESS: ML monitoring infrastructure ready")


In [ ]:
# Model Drift Detection Framework
print("Setting up model drift detection framework...")

@dataclass
class DriftDetectionConfig:
    """Configuration for drift detection"""
    drift_threshold: float = 0.05
    min_samples: int = 100
    baseline_days: int = 7
    current_days: int = 1
    significance_level: float = 0.05

class ModelDriftDetector:
    """
    Comprehensive model drift detection using statistical tests
    """
    
    def __init__(self, session, config: DriftDetectionConfig = None):
        self.session = session
        self.config = config or DriftDetectionConfig()
        
    def detect_data_drift(self, model_name: str, feature_columns: List[str]) -> Dict[str, Any]:
        """
        Detect data drift in input features using statistical tests
        """
        print(f"Detecting data drift for {model_name}...")
        
        drift_results = []
        
        # Define time periods for comparison
        current_end = datetime.datetime.now()
        current_start = current_end - datetime.timedelta(days=self.config.current_days)
        baseline_end = current_start
        baseline_start = baseline_end - datetime.timedelta(days=self.config.baseline_days)
        
        print(f"   Baseline period: {baseline_start.date()} to {baseline_end.date()}")
        print(f"   Current period: {current_start.date()} to {current_end.date()}")
        
        try:
            # Get baseline and current data
            inference_log_table = "ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.INFERENCE_REQUEST_LOG"
            
            # For demonstration, we'll check prediction drift (since we have that data)
            # In production, you'd check input feature drift
            
            baseline_predictions_sql = f'''
                SELECT PREDICTION_RESULT
                FROM {inference_log_table}
                WHERE REQUEST_TIMESTAMP BETWEEN '{baseline_start.isoformat()}' AND '{baseline_end.isoformat()}'
                AND SUCCESS_STATUS = TRUE
                LIMIT 1000
            '''
            
            current_predictions_sql = f'''
                SELECT PREDICTION_RESULT  
                FROM {inference_log_table}
                WHERE REQUEST_TIMESTAMP BETWEEN '{current_start.isoformat()}' AND '{current_end.isoformat()}'
                AND SUCCESS_STATUS = TRUE
                LIMIT 1000
            '''
            
            baseline_data = self.session.sql(baseline_predictions_sql).collect()
            current_data = self.session.sql(current_predictions_sql).collect()
            
            if len(baseline_data) < self.config.min_samples or len(current_data) < self.config.min_samples:
                print(f"   WARNING: Insufficient data for drift detection (baseline: {len(baseline_data)}, current: {len(current_data)})")
                return {'drift_detected': False, 'reason': 'Insufficient data'}
            
            # Calculate distribution statistics
            baseline_values = [row['PREDICTION_RESULT'] for row in baseline_data]
            current_values = [row['PREDICTION_RESULT'] for row in current_data]
            
            baseline_mean = sum(baseline_values) / len(baseline_values)
            current_mean = sum(current_values) / len(current_values)
            
            baseline_std = (sum((x - baseline_mean)**2 for x in baseline_values) / len(baseline_values))**0.5
            current_std = (sum((x - current_mean)**2 for x in current_values) / len(current_values))**0.5
            
            # Simple drift detection based on mean shift
            mean_shift = abs(current_mean - baseline_mean) / baseline_std if baseline_std > 0 else 0
            drift_detected = mean_shift > self.config.drift_threshold
            
            # Determine severity
            if mean_shift > 0.3:
                severity = 'CRITICAL'
            elif mean_shift > 0.2:
                severity = 'HIGH'
            elif mean_shift > 0.1:
                severity = 'MEDIUM'
            else:
                severity = 'LOW'
            
            drift_result = {
                'feature_name': 'PREDICTION_RESULT',
                'drift_score': mean_shift,
                'drift_detected': drift_detected,
                'drift_severity': severity,
                'baseline_mean': baseline_mean,
                'current_mean': current_mean,
                'baseline_std': baseline_std,
                'current_std': current_std,
                'baseline_samples': len(baseline_data),
                'current_samples': len(current_data)
            }
            
            drift_results.append(drift_result)
            
            # Log drift detection results
            self._log_drift_detection(model_name, drift_result, baseline_start, baseline_end, current_start, current_end)
            
            print(f"   Prediction drift analysis:")
            print(f"      Mean shift: {mean_shift:.4f} (threshold: {self.config.drift_threshold})")
            print(f"      Drift detected: {'Yes' if drift_detected else 'No'}")
            print(f"      Severity: {severity}")
            
        except Exception as e:
            print(f"   WARNING: Drift detection error: {e}")
            return {'drift_detected': False, 'error': str(e)}
        
        return {
            'drift_detected': any(result['drift_detected'] for result in drift_results),
            'drift_results': drift_results,
            'overall_severity': max([result['drift_severity'] for result in drift_results], key=lambda x: ['LOW', 'MEDIUM', 'HIGH', 'CRITICAL'].index(x)) if drift_results else 'LOW'
        }
    
    def _log_drift_detection(self, model_name: str, drift_result: Dict[str, Any], 
                           baseline_start: datetime.datetime, baseline_end: datetime.datetime,
                           current_start: datetime.datetime, current_end: datetime.datetime):
        """Log drift detection results"""
        
        try:
            drift_id = f"DRIFT_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_{model_name}_{drift_result['feature_name']}"
            
            drift_data = [(
                drift_id,
                model_name,
                datetime.datetime.now().isoformat(),
                'PREDICTION_DRIFT',
                drift_result['feature_name'],
                drift_result['drift_score'],
                0.05,  # Simulated p-value
                self.config.drift_threshold,
                drift_result['drift_detected'],
                drift_result['drift_severity'],
                drift_result['baseline_mean'],
                drift_result['current_mean'],
                drift_result['baseline_std'],
                drift_result['current_std'],
                0.0,  # KS statistic placeholder
                baseline_start.isoformat(),
                baseline_end.isoformat(),
                current_start.isoformat(),
                current_end.isoformat(),
                drift_result['drift_severity'] in ['HIGH', 'CRITICAL'],
                drift_result['drift_detected'],
                False
            )]
            
            drift_schema = StructType([
                StructField("DRIFT_ID", StringType()),
                StructField("MODEL_NAME", StringType()),
                StructField("DRIFT_TIMESTAMP", StringType()),
                StructField("DRIFT_TYPE", StringType()),
                StructField("FEATURE_NAME", StringType()),
                StructField("DRIFT_SCORE", DoubleType()),
                StructField("P_VALUE", DoubleType()),
                StructField("DRIFT_THRESHOLD", DoubleType()),
                StructField("DRIFT_DETECTED", BooleanType()),
                StructField("DRIFT_SEVERITY", StringType()),
                StructField("BASELINE_MEAN", DoubleType()),
                StructField("CURRENT_MEAN", DoubleType()),
                StructField("BASELINE_STD", DoubleType()),
                StructField("CURRENT_STD", DoubleType()),
                StructField("KS_STATISTIC", DoubleType()),
                StructField("BASELINE_PERIOD_START", StringType()),
                StructField("BASELINE_PERIOD_END", StringType()),
                StructField("CURRENT_PERIOD_START", StringType()),
                StructField("CURRENT_PERIOD_END", StringType()),
                StructField("REQUIRES_RETRAINING", BooleanType()),
                StructField("REQUIRES_INVESTIGATION", BooleanType()),
                StructField("ALERT_SENT", BooleanType())
            ])
            
            drift_df = self.session.create_dataframe(drift_data, schema=drift_schema)
            drift_df.write.mode("append").save_as_table("ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_MODEL_DRIFT_DETECTION")
            
        except Exception as e:
            print(f"   WARNING: Drift logging error: {e}")

# Initialize drift detector
drift_detector = ModelDriftDetector(session)

# Run drift detection
print("Running drift detection analysis...")

drift_analysis = drift_detector.detect_data_drift(
    model_name="healthcare_risk_model",
    feature_columns=["AGE", "NUM_CONDITIONS", "NUM_MEDICATIONS", "NUM_CLAIMS"]
)

if drift_analysis.get('drift_detected'):
    print(f"ALERT: Drift detected! Severity: {drift_analysis.get('overall_severity')}")
    print("   Recommended actions:")
    if drift_analysis.get('overall_severity') in ['HIGH', 'CRITICAL']:
        print("      • Consider model retraining")
        print("      • Investigate data source changes")
        print("      • Review feature engineering pipeline")
    else:
        print("      • Continue monitoring")
        print("      • Schedule deeper analysis")
else:
    print("SUCCESS: No significant drift detected - model performance stable")

print("SUCCESS: Drift detection framework operational")


In [ ]:
# Automated Alert System
print("Setting up automated alert system...")

class MLAlertManager:
    """
    Comprehensive alert management system for ML operations
    """
    
    def __init__(self, session):
        self.session = session
        self.alert_thresholds = {
            'performance': {
                'mae_degradation': 0.15,        # 15% increase in MAE
                'accuracy_drop': 0.05,          # 5% decrease in accuracy
                'response_time_increase': 0.3,   # 30% increase in response time
                'success_rate_drop': 0.02       # 2% decrease in success rate
            },
            'drift': {
                'drift_score_threshold': 0.1,   # Drift score above 0.1
                'critical_drift_threshold': 0.3 # Critical drift threshold
            },
            'volume': {
                'request_volume_drop': 0.5,     # 50% drop in requests
                'request_volume_spike': 2.0     # 200% increase in requests
            }
        }
    
    def check_performance_alerts(self) -> List[Dict[str, Any]]:
        """Check for performance-related alerts"""
        
        alerts = []
        
        try:
            # Check recent performance metrics
            performance_query = '''
                SELECT 
                    AVG(RESPONSE_TIME_MS) as avg_response_time,
                    AVG(CASE WHEN SUCCESS_STATUS THEN 1.0 ELSE 0.0 END) as success_rate,
                    COUNT(*) as request_count
                FROM ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.INFERENCE_REQUEST_LOG
                WHERE REQUEST_TIMESTAMP >= DATEADD(hour, -1, CURRENT_TIMESTAMP())
            '''
            
            current_metrics = self.session.sql(performance_query).collect()
            
            if current_metrics:
                current = current_metrics[0]
                
                # Check response time (compare to baseline of 100ms)
                baseline_response_time = 100.0
                if current['AVG_RESPONSE_TIME']:
                    response_time_increase = (current['AVG_RESPONSE_TIME'] - baseline_response_time) / baseline_response_time
                    
                    if response_time_increase > self.alert_thresholds['performance']['response_time_increase']:
                        alerts.append({
                            'alert_type': 'PERFORMANCE_DEGRADATION',
                            'alert_severity': 'WARNING' if response_time_increase < 0.5 else 'CRITICAL',
                            'alert_title': 'Response Time Degradation',
                            'alert_message': f"Response time increased by {response_time_increase:.1%} to {current['AVG_RESPONSE_TIME']:.1f}ms",
                            'metric_name': 'avg_response_time',
                            'current_value': current['AVG_RESPONSE_TIME'],
                            'threshold_value': baseline_response_time * (1 + self.alert_thresholds['performance']['response_time_increase']),
                            'baseline_value': baseline_response_time
                        })
                
                # Check success rate
                if current['SUCCESS_RATE'] < (1.0 - self.alert_thresholds['performance']['success_rate_drop']):
                    alerts.append({
                        'alert_type': 'PERFORMANCE_DEGRADATION',
                        'alert_severity': 'CRITICAL',
                        'alert_title': 'Success Rate Drop',
                        'alert_message': f"Success rate dropped to {current['SUCCESS_RATE']:.1%}",
                        'metric_name': 'success_rate',
                        'current_value': current['SUCCESS_RATE'] * 100,
                        'threshold_value': 98.0,
                        'baseline_value': 100.0
                    })
                
                # Check request volume (compare to baseline of 10 requests/hour)
                baseline_volume = 10
                volume_ratio = current['REQUEST_COUNT'] / baseline_volume if baseline_volume > 0 else 1
                
                if volume_ratio < self.alert_thresholds['volume']['request_volume_drop']:
                    alerts.append({
                        'alert_type': 'SYSTEM_ERROR',
                        'alert_severity': 'WARNING',
                        'alert_title': 'Low Request Volume',
                        'alert_message': f"Request volume dropped to {current['REQUEST_COUNT']} (expected ~{baseline_volume})",
                        'metric_name': 'request_volume',
                        'current_value': current['REQUEST_COUNT'],
                        'threshold_value': baseline_volume * self.alert_thresholds['volume']['request_volume_drop'],
                        'baseline_value': baseline_volume
                    })
                elif volume_ratio > self.alert_thresholds['volume']['request_volume_spike']:
                    alerts.append({
                        'alert_type': 'SYSTEM_ERROR',
                        'alert_severity': 'INFO',
                        'alert_title': 'High Request Volume',
                        'alert_message': f"Request volume spiked to {current['REQUEST_COUNT']} (expected ~{baseline_volume})",
                        'metric_name': 'request_volume',
                        'current_value': current['REQUEST_COUNT'],
                        'threshold_value': baseline_volume * self.alert_thresholds['volume']['request_volume_spike'],
                        'baseline_value': baseline_volume
                    })
        
        except Exception as e:
            alerts.append({
                'alert_type': 'SYSTEM_ERROR',
                'alert_severity': 'CRITICAL',
                'alert_title': 'Alert System Error',
                'alert_message': f"Error checking performance alerts: {e}",
                'metric_name': 'alert_system',
                'current_value': 0,
                'threshold_value': 1,
                'baseline_value': 1
            })
        
        return alerts
    
    def check_drift_alerts(self) -> List[Dict[str, Any]]:
        """Check for drift-related alerts"""
        
        alerts = []
        
        try:
            # Check recent drift detections
            drift_query = '''
                SELECT *
                FROM ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_MODEL_DRIFT_DETECTION
                WHERE DRIFT_TIMESTAMP >= DATEADD(hour, -24, CURRENT_TIMESTAMP())
                AND DRIFT_DETECTED = TRUE
                AND ALERT_SENT = FALSE
                ORDER BY DRIFT_TIMESTAMP DESC
            '''
            
            drift_detections = self.session.sql(drift_query).collect()
            
            for drift in drift_detections:
                severity_map = {
                    'LOW': 'INFO',
                    'MEDIUM': 'WARNING', 
                    'HIGH': 'CRITICAL',
                    'CRITICAL': 'EMERGENCY'
                }
                
                alerts.append({
                    'alert_type': 'DRIFT_DETECTED',
                    'alert_severity': severity_map.get(drift['DRIFT_SEVERITY'], 'WARNING'),
                    'alert_title': f"{drift['DRIFT_TYPE']} Detected",
                    'alert_message': f"Drift detected in {drift['FEATURE_NAME']} with score {drift['DRIFT_SCORE']:.4f}",
                    'metric_name': f"drift_{drift['FEATURE_NAME']}",
                    'current_value': drift['DRIFT_SCORE'],
                    'threshold_value': drift['DRIFT_THRESHOLD'],
                    'baseline_value': 0.0,
                    'drift_id': drift['DRIFT_ID']
                })
        
        except Exception as e:
            print(f"   WARNING: Drift alert check error: {e}")
        
        return alerts
    
    def log_alert(self, alert: Dict[str, Any], model_name: str = "healthcare_risk_model"):
        """Log alert to management system"""
        
        try:
            alert_id = f"ALERT_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_{alert['metric_name']}"
            
            alert_data = [(
                alert_id,
                datetime.datetime.now().isoformat(),
                alert['alert_type'],
                alert['alert_severity'],
                alert['alert_title'],
                alert['alert_message'],
                model_name,
                alert['metric_name'],
                alert['current_value'],
                alert['threshold_value'],
                alert['baseline_value'],
                'ACTIVE',
                None,  # acknowledged_by
                None,  # acknowledged_timestamp
                None,  # resolved_by
                None,  # resolved_timestamp
                None,  # resolution_notes
                json.dumps(['EMAIL', 'SLACK']),  # notification_channels
                False,  # notification_sent
                1      # escalation_level
            )]
            
            alert_schema = StructType([
                StructField("ALERT_ID", StringType()),
                StructField("ALERT_TIMESTAMP", StringType()),
                StructField("ALERT_TYPE", StringType()),
                StructField("ALERT_SEVERITY", StringType()),
                StructField("ALERT_TITLE", StringType()),
                StructField("ALERT_MESSAGE", StringType()),
                StructField("MODEL_NAME", StringType()),
                StructField("METRIC_NAME", StringType()),
                StructField("CURRENT_VALUE", DoubleType()),
                StructField("THRESHOLD_VALUE", DoubleType()),
                StructField("BASELINE_VALUE", DoubleType()),
                StructField("ALERT_STATUS", StringType()),
                StructField("ACKNOWLEDGED_BY", StringType()),
                StructField("ACKNOWLEDGED_TIMESTAMP", StringType()),
                StructField("RESOLVED_BY", StringType()),
                StructField("RESOLVED_TIMESTAMP", StringType()),
                StructField("RESOLUTION_NOTES", StringType()),
                StructField("NOTIFICATION_CHANNELS", StringType()),
                StructField("NOTIFICATION_SENT", BooleanType()),
                StructField("ESCALATION_LEVEL", IntegerType())
            ])
            
            alert_df = self.session.create_dataframe(alert_data, schema=alert_schema)
            alert_df.write.mode("append").save_as_table("ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_ALERT_MANAGEMENT")
            
            return alert_id
            
        except Exception as e:
            print(f"   WARNING: Alert logging error: {e}")
            return None
    
    def run_alert_check(self) -> Dict[str, Any]:
        """Run comprehensive alert check"""
        
        print("Running comprehensive alert check...")
        
        # Check all alert types
        performance_alerts = self.check_performance_alerts()
        drift_alerts = self.check_drift_alerts()
        
        all_alerts = performance_alerts + drift_alerts
        
        # Log all alerts
        logged_alerts = []
        for alert in all_alerts:
            alert_id = self.log_alert(alert)
            if alert_id:
                logged_alerts.append({**alert, 'alert_id': alert_id})
        
        # Categorize alerts by severity
        alert_summary = {
            'total_alerts': len(logged_alerts),
            'emergency': len([a for a in logged_alerts if a['alert_severity'] == 'EMERGENCY']),
            'critical': len([a for a in logged_alerts if a['alert_severity'] == 'CRITICAL']),
            'warning': len([a for a in logged_alerts if a['alert_severity'] == 'WARNING']),
            'info': len([a for a in logged_alerts if a['alert_severity'] == 'INFO']),
            'alerts': logged_alerts
        }
        
        return alert_summary

# Initialize alert manager
alert_manager = MLAlertManager(session)

# Run alert check
print("Running alert system check...")

alert_summary = alert_manager.run_alert_check()

print(f"Alert Summary:")
print(f"   Total alerts: {alert_summary['total_alerts']}")

if alert_summary['total_alerts'] > 0:
    print(f"   Emergency: {alert_summary['emergency']}")
    print(f"   Critical: {alert_summary['critical']}")
    print(f"   Warning: {alert_summary['warning']}")
    print(f"   Info: {alert_summary['info']}")
    
    print(f"\nActive Alerts:")
    for alert in alert_summary['alerts']:
        severity_prefix = {'EMERGENCY': 'EMERGENCY', 'CRITICAL': 'CRITICAL', 'WARNING': 'WARNING', 'INFO': 'INFO'}
        print(f"   {severity_prefix.get(alert['alert_severity'], 'ALERT')} - {alert['alert_title']}: {alert['alert_message']}")
else:
    print("   SUCCESS: No alerts - system operating normally")

print("SUCCESS: Automated alert system operational")


In [ ]:
# Business Impact & KPI Monitoring
print("Setting up business impact and KPI monitoring...")

class BusinessImpactMonitor:
    """
    Monitor business impact and KPIs of the ML system
    """
    
    def __init__(self, session):
        self.session = session
    
    def calculate_clinical_impact_metrics(self, period_days: int = 7) -> Dict[str, Any]:
        """Calculate clinical impact metrics over a specified period"""
        
        print(f"Calculating clinical impact metrics for last {period_days} days...")
        
        period_start = datetime.datetime.now() - datetime.timedelta(days=period_days)
        period_end = datetime.datetime.now()
        
        try:
            # Get prediction data from multiple sources
            predictions_query = f'''
                SELECT 
                    COUNT(*) as total_patients_assessed,
                    SUM(CASE WHEN RISK_CATEGORY = 'HIGH' THEN 1 ELSE 0 END) as high_risk_identified,
                    SUM(CASE WHEN RISK_CATEGORY = 'MEDIUM' THEN 1 ELSE 0 END) as medium_risk_identified,
                    SUM(CASE WHEN RISK_CATEGORY = 'LOW' THEN 1 ELSE 0 END) as low_risk_identified,
                    AVG(RISK_SCORE) as average_risk_score,
                    STDDEV(RISK_SCORE) as risk_score_std
                FROM ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.BATCH_INFERENCE_RESULTS
                WHERE INFERENCE_TIMESTAMP >= '{period_start.isoformat()}'
                AND INFERENCE_TIMESTAMP <= '{period_end.isoformat()}'
            '''
            
            prediction_stats = self.session.sql(predictions_query).collect()
            
            if prediction_stats and prediction_stats[0]['TOTAL_PATIENTS_ASSESSED']:
                stats = prediction_stats[0]
                
                # Calculate derived metrics
                high_risk_percentage = (stats['HIGH_RISK_IDENTIFIED'] / stats['TOTAL_PATIENTS_ASSESSED']) * 100
                
                # Estimate clinical interventions (assuming 80% of high-risk cases trigger intervention)
                estimated_interventions = int(stats['HIGH_RISK_IDENTIFIED'] * 0.8)
                
                # Estimate potential adverse events prevented (based on literature: 15-25% reduction)
                estimated_events_prevented = int(stats['HIGH_RISK_IDENTIFIED'] * 0.2)
                
                # Estimate cost savings ($5,000 per prevented adverse event, $500 per early intervention)
                cost_savings = (estimated_events_prevented * 5000) + (estimated_interventions * 500)
                
                clinical_metrics = {
                    'period_start': period_start.isoformat(),
                    'period_end': period_end.isoformat(),
                    'patients_risk_assessed': stats['TOTAL_PATIENTS_ASSESSED'],
                    'high_risk_patients_identified': stats['HIGH_RISK_IDENTIFIED'],
                    'medium_risk_patients_identified': stats['MEDIUM_RISK_IDENTIFIED'],
                    'low_risk_patients_identified': stats['LOW_RISK_IDENTIFIED'],
                    'high_risk_percentage': high_risk_percentage,
                    'average_risk_score': float(stats['AVERAGE_RISK_SCORE']),
                    'risk_score_std': float(stats['RISK_SCORE_STD']),
                    'clinical_interventions_triggered': estimated_interventions,
                    'potential_adverse_events_prevented': estimated_events_prevented,
                    'estimated_cost_savings': cost_savings
                }
                
                print(f"   Patients assessed: {clinical_metrics['patients_risk_assessed']:,}")
                print(f"   High-risk identified: {clinical_metrics['high_risk_patients_identified']:,} ({high_risk_percentage:.1f}%)")
                print(f"   Clinical interventions: {estimated_interventions:,}")
                print(f"   Adverse events prevented: {estimated_events_prevented:,}")
                print(f"   Estimated cost savings: ${cost_savings:,.2f}")
                
                return clinical_metrics
            else:
                print("   WARNING: No prediction data available for the specified period")
                return {'patients_risk_assessed': 0, 'error': 'No data available'}
                
        except Exception as e:
            print(f"   WARNING: Clinical impact calculation error: {e}")
            return {'error': str(e)}
    
    def calculate_operational_metrics(self) -> Dict[str, Any]:
        """Calculate operational efficiency metrics"""
        
        print("Calculating operational efficiency metrics...")
        
        try:
            # Get inference performance metrics
            performance_query = '''
                SELECT 
                    COUNT(*) as total_requests,
                    AVG(RESPONSE_TIME_MS) as avg_response_time,
                    AVG(CASE WHEN SUCCESS_STATUS THEN 1.0 ELSE 0.0 END) as success_rate,
                    SUM(CASE WHEN SUCCESS_STATUS THEN 1 ELSE 0 END) as successful_requests
                FROM ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.INFERENCE_REQUEST_LOG
                WHERE REQUEST_TIMESTAMP >= DATEADD(day, -7, CURRENT_TIMESTAMP())
            '''
            
            perf_stats = self.session.sql(performance_query).collect()
            
            if perf_stats:
                stats = perf_stats[0]
                
                # Calculate efficiency metrics
                # Assume manual risk assessment takes 15 minutes per patient
                manual_time_per_patient = 15  # minutes
                automated_time_per_patient = (stats['AVG_RESPONSE_TIME'] or 100) / 1000 / 60  # convert ms to minutes
                
                time_savings_per_patient = manual_time_per_patient - automated_time_per_patient
                total_time_saved = time_savings_per_patient * (stats['SUCCESSFUL_REQUESTS'] or 0)
                
                # Efficiency improvement percentage
                efficiency_improvement = (time_savings_per_patient / manual_time_per_patient) * 100
                
                operational_metrics = {
                    'total_inference_requests': stats['TOTAL_REQUESTS'] or 0,
                    'successful_requests': stats['SUCCESSFUL_REQUESTS'] or 0,
                    'success_rate_percentage': (stats['SUCCESS_RATE'] or 0) * 100,
                    'average_response_time_ms': stats['AVG_RESPONSE_TIME'] or 0,
                    'time_saved_hours': total_time_saved / 60,
                    'efficiency_improvement_percentage': efficiency_improvement,
                    'manual_time_per_assessment_minutes': manual_time_per_patient,
                    'automated_time_per_assessment_minutes': automated_time_per_patient
                }
                
                print(f"   Total requests: {operational_metrics['total_inference_requests']:,}")
                print(f"   Success rate: {operational_metrics['success_rate_percentage']:.1f}%")
                print(f"   Avg response time: {operational_metrics['average_response_time_ms']:.1f}ms")
                print(f"   Time saved: {operational_metrics['time_saved_hours']:.1f} hours")
                print(f"   Efficiency improvement: {efficiency_improvement:.1f}%")
                
                return operational_metrics
            else:
                return {'error': 'No performance data available'}
                
        except Exception as e:
            print(f"   WARNING: Operational metrics calculation error: {e}")
            return {'error': str(e)}
    
    def log_business_impact(self, clinical_metrics: Dict[str, Any], operational_metrics: Dict[str, Any]):
        """Log business impact metrics"""
        
        try:
            impact_id = f"IMPACT_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
            
            # Calculate false positive/negative estimates (simulated for demo)
            false_positive_rate = 0.15  # 15% estimated false positive rate
            false_negative_rate = 0.08   # 8% estimated false negative rate
            clinical_accuracy_score = 0.85  # 85% clinical accuracy feedback
            
            impact_data = [(
                impact_id,
                datetime.datetime.now().isoformat(),
                clinical_metrics.get('patients_risk_assessed', 0),
                clinical_metrics.get('high_risk_patients_identified', 0),
                clinical_metrics.get('clinical_interventions_triggered', 0),
                clinical_metrics.get('potential_adverse_events_prevented', 0),
                clinical_metrics.get('estimated_cost_savings', 0.0),
                operational_metrics.get('efficiency_improvement_percentage', 0.0),
                operational_metrics.get('time_saved_hours', 0.0),
                false_positive_rate,
                false_negative_rate,
                clinical_accuracy_score,
                clinical_metrics.get('period_start'),
                clinical_metrics.get('period_end'),
                'WEEKLY'
            )]
            
            impact_schema = StructType([
                StructField("IMPACT_ID", StringType()),
                StructField("MONITORING_TIMESTAMP", StringType()),
                StructField("PATIENTS_RISK_ASSESSED", IntegerType()),
                StructField("HIGH_RISK_PATIENTS_IDENTIFIED", IntegerType()),
                StructField("CLINICAL_INTERVENTIONS_TRIGGERED", IntegerType()),
                StructField("POTENTIAL_ADVERSE_EVENTS_PREVENTED", IntegerType()),
                StructField("COST_SAVINGS_ESTIMATED", DoubleType()),
                StructField("EFFICIENCY_IMPROVEMENT_PERCENTAGE", DoubleType()),
                StructField("STAFF_TIME_SAVED_HOURS", DoubleType()),
                StructField("FALSE_POSITIVE_RATE", DoubleType()),
                StructField("FALSE_NEGATIVE_RATE", DoubleType()),
                StructField("CLINICAL_ACCURACY_FEEDBACK_SCORE", DoubleType()),
                StructField("MEASUREMENT_PERIOD_START", StringType()),
                StructField("MEASUREMENT_PERIOD_END", StringType()),
                StructField("REPORTING_FREQUENCY", StringType())
            ])
            
            impact_df = self.session.create_dataframe(impact_data, schema=impact_schema)
            impact_df.write.mode("append").save_as_table("ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.ML_BUSINESS_IMPACT_MONITORING")
            
            print(f"   SUCCESS: Business impact logged: {impact_id}")
            
        except Exception as e:
            print(f"   WARNING: Business impact logging error: {e}")

# Initialize business impact monitor
impact_monitor = BusinessImpactMonitor(session)

# Calculate and log business impact
print("Running business impact analysis...")

clinical_impact = impact_monitor.calculate_clinical_impact_metrics(period_days=7)
operational_impact = impact_monitor.calculate_operational_metrics()

# Log business impact
if not clinical_impact.get('error') and not operational_impact.get('error'):
    impact_monitor.log_business_impact(clinical_impact, operational_impact)
    
    print(f"\nBusiness Impact Summary:")
    print(f"   Clinical Impact:")
    print(f"      • Patients assessed: {clinical_impact.get('patients_risk_assessed', 0):,}")
    print(f"      • High-risk identified: {clinical_impact.get('high_risk_patients_identified', 0):,}")
    print(f"      • Interventions triggered: {clinical_impact.get('clinical_interventions_triggered', 0):,}")
    print(f"      • Adverse events prevented: {clinical_impact.get('potential_adverse_events_prevented', 0):,}")
    print(f"      • Cost savings: ${clinical_impact.get('estimated_cost_savings', 0):,.2f}")
    print(f"   Operational Impact:")
    print(f"      • Efficiency improvement: {operational_impact.get('efficiency_improvement_percentage', 0):.1f}%")
    print(f"      • Time saved: {operational_impact.get('time_saved_hours', 0):.1f} hours")
    print(f"      • Success rate: {operational_impact.get('success_rate_percentage', 0):.1f}%")
else:
    print("   WARNING: Insufficient data for complete business impact analysis")

print("SUCCESS: Business impact monitoring operational")


In [ ]:
# Native Snowflake Model Monitor Integration
print("Integrating native Snowflake Model Monitors with observability platform...")

def setup_native_model_monitors():
    """
    Set up native Snowflake Model Monitors for comprehensive model observability
    """
    print("Setting up native Snowflake Model Monitors...")
    
    # First, create monitoring data structure compatible with Model Monitors
    print("Creating Model Monitor compatible logging table...")
    
    monitor_table_sql = """
        CREATE TABLE IF NOT EXISTS ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.HEALTHCARE_INFERENCE_MONITOR_LOGS (
            ID STRING,
            TIMESTAMP TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
            
            -- Model Features (input data)
            AGE NUMBER,
            NUM_CONDITIONS NUMBER,
            NUM_MEDICATIONS NUMBER,
            NUM_CLAIMS NUMBER,
            
            -- Predictions
            PREDICTION_SCORE NUMBER,
            PREDICTION_CLASS STRING,
            
            -- Ground Truth (when available)
            ACTUAL_RISK_SCORE NUMBER,
            ACTUAL_OUTCOME STRING,
            
            -- Metadata
            MODEL_VERSION STRING,
            INFERENCE_METHOD STRING,
            PATIENT_ID STRING
        )
    """
    
    try:
        session.sql(monitor_table_sql).collect()
        print("   SUCCESS: Monitor logging table created")
        
        # Populate with sample data for monitoring
        sample_data_sql = """
            INSERT INTO ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.HEALTHCARE_INFERENCE_MONITOR_LOGS
            (ID, AGE, NUM_CONDITIONS, NUM_MEDICATIONS, NUM_CLAIMS, 
             PREDICTION_SCORE, PREDICTION_CLASS, MODEL_VERSION, INFERENCE_METHOD, PATIENT_ID)
            SELECT 
                'MON_' || ROW_NUMBER() OVER (ORDER BY RANDOM()) as ID,
                UNIFORM(25, 85, RANDOM()) as AGE,
                UNIFORM(1, 15, RANDOM()) as NUM_CONDITIONS,
                UNIFORM(1, 20, RANDOM()) as NUM_MEDICATIONS,
                UNIFORM(5, 50, RANDOM()) as NUM_CLAIMS,
                UNIFORM(0, 100, RANDOM()) as PREDICTION_SCORE,
                CASE 
                    WHEN UNIFORM(0, 100, RANDOM()) < 30 THEN 'LOW'
                    WHEN UNIFORM(0, 100, RANDOM()) < 70 THEN 'MEDIUM'
                    ELSE 'HIGH'
                END as PREDICTION_CLASS,
                'HEALTHCARE_RISK_XGBOOST_V1' as MODEL_VERSION,
                'INFERENCE_SERVICE' as INFERENCE_METHOD,
                'PAT_' || ROW_NUMBER() OVER (ORDER BY RANDOM()) as PATIENT_ID
            FROM TABLE(GENERATOR(ROWCOUNT => 500))
        """
        
        session.sql(sample_data_sql).collect()
        print("   SUCCESS: Sample monitoring data populated (500 records)")
        
    except Exception as e:
        print(f"   WARNING: Monitor table setup error: {e}")
    
    # Check available models for monitor setup
    try:
        models_check = session.sql("SHOW MODELS IN SCHEMA ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS;").collect()
        
        if models_check:
            print(f"\nAvailable models for monitoring:")
            for model in models_check:
                print(f"   • {model['name']} (created: {model['created_on']})")
            
            # Focus on regression models (supported by Model Monitors)
            regression_models = [
                'HEALTHCARE_RISK_XGBOOST_REGRESSOR',
                'HEALTHCARE_RISK_SCORE_REGRESSOR', 
                'HEALTHCARE_RISK_PREDICTOR'
            ]
            
            monitor_created = False
            for model_name in regression_models:
                try:
                    versions_query = f"SHOW VERSIONS IN MODEL ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.{model_name};"
                    versions = session.sql(versions_query).collect()
                    
                    if versions:
                        latest_version = versions[0]['name']
                        print(f"\nCreating Model Monitor for {model_name}...")
                        print(f"   Using version: {latest_version}")
                        
                        # Create Model Monitor (using supported regression model)
                        monitor_sql = f"""
CREATE MODEL MONITOR IF NOT EXISTS HEALTHCARE_INFERENCE_MONITOR WITH
    MODEL = ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.{model_name}
    VERSION = '{latest_version}'
    FUNCTION = 'predict'
    SOURCE = ADVERSE_EVENT_MONITORING.DEMO_ANALYTICS.HEALTHCARE_INFERENCE_MONITOR_LOGS
    WAREHOUSE = ADVERSE_EVENT_WH
    REFRESH_INTERVAL = '1 day'
    AGGREGATION_WINDOW = '1 day'
    TIMESTAMP_COLUMN = TIMESTAMP
    ID_COLUMNS = ('ID')
    PREDICTION_SCORE_COLUMNS = ('PREDICTION_SCORE')
                        """
                        
                        try:
                            session.sql(monitor_sql).collect()
                            print(f"   SUCCESS: Model Monitor created successfully!")
                            print(f"   Monitor Name: HEALTHCARE_INFERENCE_MONITOR")
                            print(f"   Monitoring: {model_name} (version: {latest_version})")
                            print(f"   Task Type: TABULAR_REGRESSION (supported)")
                            monitor_created = True
                            break
                            
                        except Exception as e:
                            print(f"   WARNING: Monitor creation failed for {model_name}: {e}")
                            continue
                            
                except Exception as e:
                    print(f"   WARNING: Could not check versions for {model_name}: {e}")
                    continue
            
            if not monitor_created:
                print("WARNING: Could not create Model Monitor with available models")
                print("INFO: Manual setup may be required in Snowsight UI")
                
        else:
            print("FAILED: No models found - please run notebook 05 (Model Training) first")
            
    except Exception as e:
        print(f"WARNING: Model check error: {e}")

def check_model_monitor_status():
    """Check the status of created Model Monitors"""
    
    print("\nChecking Model Monitor status...")
    
    try:
        # Show all model monitors
        monitors_sql = "SHOW MODEL MONITORS;"
        monitors = session.sql(monitors_sql).collect()
        
        if monitors:
            print(f"SUCCESS: Found {len(monitors)} Model Monitor(s):")
            for monitor in monitors:
                try:
                    name = monitor.get('name', monitor.get('NAME', 'Unknown'))
                    model_name = monitor.get('model_name', monitor.get('MODEL_NAME', 'Unknown'))
                    status = monitor.get('status', monitor.get('STATUS', 'Unknown'))
                    print(f"   {name}: {model_name} ({status})")
                except Exception as e:
                    print(f"   Monitor found but details unavailable: {e}")
        else:
            print("FAILED: No Model Monitors found")
            
        # Try to get specific monitor details
        try:
            describe_sql = "DESCRIBE MODEL MONITOR HEALTHCARE_INFERENCE_MONITOR;"
            details = session.sql(describe_sql).collect()
            
            if details:
                print("\nHealthcare Monitor Details:")
                for detail in details:
                    try:
                        prop = detail.get('property', detail.get('PROPERTY', 'Unknown'))
                        value = detail.get('value', detail.get('VALUE', 'Unknown'))
                        print(f"   • {prop}: {value}")
                    except:
                        print(f"   • Detail: {detail}")
            else:
                print("\nINFO: Monitor details not available yet")
                
        except Exception as e:
            print(f"\nINFO: Healthcare monitor details: {e}")
            
    except Exception as e:
        print(f"WARNING: Could not check monitor status: {e}")

def demonstrate_monitor_queries():
    """Demonstrate native Model Monitor query capabilities"""
    
    print("\nModel Monitor Query Capabilities...")
    
    # Example queries (will work once monitor is operational)
    monitor_queries = {
        "Drift Metrics": """
            SELECT *
            FROM TABLE(MODEL_MONITOR_DRIFT_METRIC (
                'HEALTHCARE_INFERENCE_MONITOR',
                'PSI',  -- Population Stability Index
                'AGE',
                'DAILY',
                DATEADD('day', -7, CURRENT_TIMESTAMP()),
                CURRENT_TIMESTAMP()
            ))
            LIMIT 5
        """,
        "Performance Metrics": """
            SELECT *
            FROM TABLE(MODEL_MONITOR_PERFORMANCE_METRIC (
                'HEALTHCARE_INFERENCE_MONITOR',
                'ACCURACY',
                'DAILY', 
                DATEADD('day', -7, CURRENT_TIMESTAMP()),
                CURRENT_TIMESTAMP()
            ))
            LIMIT 5
        """,
        "Statistical Metrics": """
            SELECT *
            FROM TABLE(MODEL_MONITOR_STAT_METRIC (
                'HEALTHCARE_INFERENCE_MONITOR',
                'COUNT',
                'DAILY',
                DATEADD('day', -7, CURRENT_TIMESTAMP()), 
                CURRENT_TIMESTAMP()
            ))
            LIMIT 5
        """
    }
    
    print("Available Monitor Queries:")
    for query_type, query in monitor_queries.items():
        print(f"\n**{query_type}:**")
        print("```sql")
        print(query.strip())
        print("```")
        
        # Try to execute (will fail until monitor is fully operational)
        try:
            result = session.sql(query).collect()
            if result:
                print(f"SUCCESS: {query_type} query successful - {len(result)} records")
            else:
                print(f"INFO: {query_type} query returned no data")
        except Exception as e:
            print(f"INFO: {query_type} not available yet: Monitor initializing")

# Set up native Snowflake Model Monitors
setup_native_model_monitors()

# Check monitor status  
check_model_monitor_status()

# Demonstrate query capabilities
demonstrate_monitor_queries()

print(f"\nNative Model Monitor Benefits:")
print(f"   Automated Drift Detection: Population Stability Index, KL divergence")
print(f"   Performance Tracking: Accuracy, precision, recall metrics")
print(f"   Built-in Alerting: Threshold-based notifications")
print(f"   Snowsight Integration: Visual dashboards and reports")
print(f"   Real-time Monitoring: Continuous model health tracking")
print(f"   Compliance Ready: Audit trails and regulatory reporting")

print(f"\nIntegration with Observability Platform:")
print(f"   Combines with existing tables: ML_MODEL_DRIFT_DETECTION, ML_ALERT_MANAGEMENT")
print(f"   Enhances drift detection: Native + statistical analysis")
print(f"   Unified dashboards: Executive + technical monitoring")
print(f"   Comprehensive alerting: Business + technical alerts")

print(f"\nAccess Your Monitors:")
print(f"   Snowsight: AI & ML → Models → [Model Name] → Monitors")
print(f"   SQL Queries: Use MODEL_MONITOR_* functions")
print(f"   Management: Configure thresholds and alerts")

print(f"\nSUCCESS: Native Snowflake Model Monitors integrated with ML Observability Platform!")
